# Bagged Trees and Random Forests

#### 🎯 Learning Goals

1. Understand the concept of **Ensemble Methods**, why they are useful, and how they work.
2. Learn how to implement **Random Forests** in `scikit-learn`.

In [1]:
# Load our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use a nicer style for plots
plt.style.use("seaborn-v0_8-muted")

___

## Ensemble Methods

> *Sometimes, the whole is greater than the sum of its parts.*

This quote is a good description of the key idea behind **Ensemble Methods**. Instead of having a single predictor do the job, we can take an ensemble of relatively *weak* predictors and have them work together to make better predictions than any single predictor could.

We have learned about cross-validation to make our predictors more robust and drastically improve their generalization to unseen data.

### Bagging

Bagging, short for "bootstrap aggregating," is a technique designed to improve the stability and accuracy of machine learning algorithms by reducing their variance. This approach is particularly beneficial for decision trees, which are known to be high variance, low bias estimators. Decision trees are prone to varying greatly with small changes in the training data; resampling the data often leads to quite different trees and predictions.

Consider $B$ random variables $Z_1, Z_2, \dots, Z_B$, each independently drawn from a distribution with variance $\sigma^2$. The variance of the average $\bar{Z}$ of these variables is derived as follows:

$$
\begin{align*}
\text{Var}(\bar{Z}) &= \text{Var}\left(\frac{1}{B}\sum_{i=1}^B Z_i\right) \\
&= \frac{1}{B^2}\text{Var}\left(\sum_{i=1}^B Z_i\right) \\
&= \frac{1}{B^2}\sum_{i=1}^B \text{Var}(Z_i) \text{ since the } Z_i \text{'s are independent} \\
&= \frac{1}{B^2} \cdot B\sigma^2 \text{ because each } Z_i \text{ has variance } \sigma^2 \\
&= \frac{\sigma^2}{B}.
\end{align*}
$$

This mathematical result sheds light on the bagging process. If we create $B$ different training datasets via bootstrapping and train $B$ individual models, denoted as $\hat{f}^{*b}(x)$ for the $b^\text{th}$ model using the $b^\text{th}$ dataset, then averaging the output of these models will yield a combined predictor with reduced variance. The intuition behind this is that different models will likely make different errors on various segments of the data, and by averaging their outputs, we can cancel out some of these errors, hence lowering the overall prediction error.

When bagging, we typically don't have access to multiple training datasets. Instead, we create $B$ different datasets via [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)), and train $B$ individual models using these datasets. The final prediction is then made by averaging the predictions of the $B$ individual models. This procedure is known as **bagging**.

In [2]:
# Load our data
housing = pd.read_csv("data/ames_housing.csv")
housing.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,...,fence,misc_feature,misc_val,mo_sold,year_sold,sale_type,sale_condition,sale_price,longitude,latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,None,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,None,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,None,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,None,0,3,2010,WD,Normal,189900,-93.638933,42.060899


In [3]:
# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split
# Import the mean_squared_error function under the alias mse
from sklearn.metrics import mean_squared_error as mse
# Import the resampling helper
from sklearn.utils import resample

In [4]:
# Split our data intro features and targets
y = np.log(housing["sale_price"]) # Use the logarithm of the sale price
features = ["lot_frontage", "lot_area", "year_built", "pool_area"]
X = housing[features]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=72)

In [5]:
# Fit a single tree to our test
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [6]:
# Compute the training and test mse
y_pred_train = tree.predict(X_train)
y_pred_test = tree.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train))
print("Test MSE : ", mse(y_test, y_pred_test))

Train MSE:  0.001216739449040038
Test MSE :  0.1105799918505865


In [7]:
# Let us now proceed with bagging, using B = 100 trees
B = 100 # Number of estimators to bag

# Create matrices to hold the predictions (n x B), each row is an observation,
# each column is a tree, i.e., in row 1, we have the predictions of the B=100 
# trees for the first observation
y_pred_train_bag = np.zeros((X_train.shape[0], B))
y_pred_test_bag = np.zeros((X_test.shape[0], B))

# Iterate over the B estimators
for b in range(B):
    tree_b = DecisionTreeRegressor()

    # Sample with replacement from X_train / y_train
    X_train_b, y_train_b = resample(X_train, y_train)
    
    # Train the tree on the bootstrapped sample
    tree_b.fit(X_train_b, y_train_b)

    # Predict on X_train and X_test
    y_pred_train_bag[:, b] = tree_b.predict(X_train)
    y_pred_test_bag[:, b] = tree_b.predict(X_test)

# Take the average of the predictions over all trees
y_pred_train_bag = y_pred_train_bag.mean(axis=1)
y_pred_test_bag = y_pred_test_bag.mean(axis=1)

print("Train MSE: ", mse(y_train, y_pred_train_bag))
print("Test MSE : ", mse(y_test, y_pred_test_bag))

Train MSE:  0.010115499129406413
Test MSE :  0.07430649558887976


Of course, implementing bagging from scratch is simple and a nice didactic example, but in practice, `scikit-learn` has a class to implement either `BaggingRegressor` or `BaggingClassifier` for us. Depending on whether we want to tacke a regression or classification problem.

In [8]:
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

In [9]:
# Create a bagged tree estimator with B=100 trees
bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=B)

# Fit the bagged estimator and compute the MSE on the training set
bagged_trees.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_bag = bagged_trees.predict(X_train)
y_pred_test_bag = bagged_trees.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_bag))
print("Test MSE : ", mse(y_test, y_pred_test_bag))

Train MSE:  0.010052876040873814
Test MSE :  0.07372084154112407


Which is the same as we obtained before (albeit some small differences due to the random nature of the algorithm). Bagging is a powerful technique that can be applied to any kind of predictor, but it is especially useful for decision trees.

#### ➡️ ✏️ Task 1

1. Train a `LinearRegression` model on the train data and evaluate its performance (both on train and test data).
2. Fllowing the example above, train a `BaggingRegressor` model that uses `LinearRegression` as its base estimator. Evaluate its performance (both on train and test data).

What do you observe? Did the performance improve? Why? Why not? Compare the results with the `DecisionTreeRegressor` and its bagged version. Discuss with your classmates why you think there are differences between the two.

In [10]:
# Import the linear regression model
from sklearn.linear_model import LinearRegression

In [11]:
# ➡️ ✏️ Your code here

In [12]:
# TODO: REMOVE SOLUTION

linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_lin = linreg.predict(X_train)
y_pred_test_lin = linreg.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_lin))
print("Test MSE : ", mse(y_test, y_pred_test_lin))

Train MSE:  0.09125266962955848
Test MSE :  0.09157895171291991


In [13]:
# TODO: REMOVE SOLUTION

bagged_regression = BaggingRegressor(LinearRegression(), n_estimators=B)

# Fit the bagged estimator and compute the MSE on the training set
bagged_regression.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_bag = bagged_regression.predict(X_train)
y_pred_test_bag = bagged_regression.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_bag))
print("Test MSE : ", mse(y_test, y_pred_test_bag))

Train MSE:  0.09136241615123851
Test MSE :  0.09226023822450422


___

## Random Forests

Random forests provide an extension of bagged trees with an additional layer of randomness compared to generic bagging. Like bagging, random forests involve building multiple decision trees on bootstrapped training samples. The key difference lies in **how the trees are constructed**. In bagging, each tree is built on a bootstrapped sample of the training data. In random forests, each tree is built on a bootstrapped sample of the training data **with a random subset of features**. 

In essence, the trees in a random forest do not have the same information available to them when they are being built, many of them being trained on a different subset of features. This introduces further randomization into the model, which tends to reduce variance even further and improve performance.

More formally, suppose we have a feature space $\mathcal{X} \in \mathbb{R}^p$. Each tree in the bagged trees is fitted using the $p$ features, while, in a random forest, each tree is only given a random subset of $m$ features, where $m < p$. The value of $m$ is specified by the user and is held constant throughout the fitting process. A common choice is $m \approx \sqrt{p}$. Notice that if we set $m = p$, the random forest and bagged trees are equivalent.

In [14]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

In [15]:
rf = RandomForestRegressor(n_estimators=B)
rf.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_rf))
print("Test MSE : ", mse(y_test, y_pred_test_rf))

Train MSE:  0.010355853107657518
Test MSE :  0.07403159275256627


#### ➡️ ✏️ Task 2

Do you have an intuition as to why the random forest is not working better than the bagged trees in this case? What would you do to improve the performance of the random forest?

#### ➡️ ✏️ Task 3

+ Choose more features from the initial dataset (at least 30). If need be, use dummies where appropriate.
+ Construct a `RandomForestRegressor` and a `BaggingRegressor` using `DecisionTreeRegressor` as the base estimator.
+ Fit the two estimators to your new dataset, compare their performance on train and test sets. What do you observe?

In [20]:
# ➡️ ✏️ Your code here

In [35]:
# TODO: REMOVE SOLUTION

features = housing.columns.drop("sale_price")
# Take all the features that are not categorical (strings, i.e., 'object' dtype)
categorical_features = [f for f in features if housing[f].dtype == "object"]

X = housing[features]
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)
y = np.log(housing["sale_price"])

In [36]:
# TODO: REMOVE SOLUTION

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=72)

# Fit the random forest and bagged trees
rf = RandomForestRegressor(n_estimators=B)
rf.fit(X_train, y_train)

bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=B)
bagged_trees.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

y_pred_train_bag = bagged_trees.predict(X_train)
y_pred_test_bag = bagged_trees.predict(X_test)

print("Train MSE (RF): ", mse(y_train, y_pred_train_rf))
print("Test MSE (RF) : ", mse(y_test, y_pred_test_rf))

print("Train MSE (Bagged Trees): ", mse(y_train, y_pred_train_bag))
print("Test MSE (Bagged Trees) : ", mse(y_test, y_pred_test_bag))

Train MSE (RF):  0.0028244672953999204
Test MSE (RF) :  0.023991012542825717
Train MSE (Bagged Trees):  0.0028032839173379506
Test MSE (Bagged Trees) :  0.024238494811091475
